Файл содержит реализацию модели векторных представлений слов (Word2Vec) с использованием двух подходов: **CBOW (Continuous Bag of Words)** и **Skip-gram Negative Sampling (SGNS)**.

---

### **1. Что такое Word2Vec?**
Word2Vec — это метод для представления слов в виде векторов (эмбеддингов), которые захватывают семантические и синтаксические свойства слов. Основная идея заключается в том, что слова, которые часто встречаются в похожих контекстах, будут иметь схожие векторные представления.

Word2Vec может быть реализован двумя способами:
1. **CBOW (Continuous Bag of Words)**: предсказывает центральное слово на основе контекста (окружающих слов).
2. **Skip-gram**: предсказывает контекстные слова на основе центрального слова.

---

### **2. Основные этапы решения задачи**

#### **2.1. Подготовка данных**
Перед обучением модели текст нужно обработать:
- **Токенизация**: Разбиваем текст на предложения и слова. Убираем пунктуацию и приводим слова к нижнему регистру. Опционально можно использовать лемматизацию (приведение слов к начальной форме).
- **Построение словаря**: Создаем словарь, который сопоставляет каждому слову уникальный индекс. Также подсчитываем частоты слов, чтобы использовать их для негативного сэмплирования.
- **Генерация пар (центральное слово, контекст)**: Для каждого слова в предложении создаем пары (центральное слово, контекстное слово) в заданном окне (например, 2 слова слева и 2 слова справа).

В коде это реализовано функциями:
- `tokenize_paragrah`: токенизация текста.
- `build_vocab`: создание словаря и частотной таблицы.
- `generate_tokens_pairs`: генерация пар (центральное слово, контекстное слово).

#### **2.2. Негативное сэмплирование**
Skip-gram Negative Sampling (SGNS) использует негативное сэмплирование для ускорения обучения. Вместо того чтобы предсказывать вероятности для всех слов в словаре, модель обучается отличать правильные пары (центральное слово, контекстное слово) от случайных пар (негативных примеров).

В коде это реализовано функцией:
- `get_negative_samples`: выбирает случайные слова из словаря на основе их частот.

#### **2.3. Построение обучающего набора**
Для каждой пары (центральное слово, контекстное слово) создаются:
- One-hot векторы для центрального слова и контекстного слова.
- Негативные примеры (слова, которые не являются контекстом).

В коде это реализовано функцией:
- `build_training_set`: создает обучающий набор.

#### **2.4. Реализация моделей**
В коде реализованы две модели:
1. **CBOW**: Использует два слоя (матрицы весов). Центральное слово предсказывается на основе суммы векторов контекстных слов.
2. **SGNS**: Использует два слоя (матрицы весов). Центральное слово используется для предсказания контекстных слов, а также для минимизации ошибки на негативных примерах.

Модели реализованы в классах:
- `CBOW`: реализует Continuous Bag of Words.
- `SGNS`: реализует Skip-gram Negative Sampling.

#### **2.5. Обучение моделей**
Модели обучаются с использованием градиентного спуска. Для CBOW используется функция потерь `CrossEntropyLoss`, а для SGNS — логарифмическая сигмоида (`logsigmoid`).

Обучение реализовано в циклах:
- Для CBOW: предсказания сравниваются с истинными значениями, и веса обновляются.
- Для SGNS: минимизируется ошибка на положительных и негативных примерах.

#### **2.6. Поиск похожих слов**
После обучения модели можно использовать эмбеддинги для поиска похожих слов. Для этого вычисляется **косинусное сходство** между векторами слов:
\[
\text{cosine\_similarity}(v_1, v_2) = \frac{v_1 \cdot v_2}{\|v_1\| \cdot \|v_2\|}
\]
Слова с наибольшим сходством считаются наиболее похожими.

В коде это реализовано функциями:
- `cosine_similarity`: вычисляет косинусное сходство.
- `most_similar`: находит наиболее похожие слова для заданного слова.

---

### **3. Концептуальное объяснение кода**

#### **3.1. Импорт библиотек**
- `regex`, `numpy`, `torch`: для работы с текстом, массивами и нейросетями.
- `spacy`: для токенизации текста.
- `plotly`: для визуализации ошибки обучения.

#### **3.2. Обработка текста**
1. Текст загружается из файла `ml_text.txt`.
2. Текст токенизируется на предложения и слова с помощью `spacy`.
3. Создается словарь, который сопоставляет каждому слову уникальный индекс. Также подсчитываются частоты слов.

#### **3.3. Генерация пар (центральное слово, контекст)**
Для каждого слова в предложении создаются пары (центральное слово, контекстное слово) в заданном окне. Например:
- Предложение: "I love programming."
- Пары: `("I", "love")`, `("love", "I")`, `("love", "programming")`, `("programming", "love")`.

#### **3.4. Построение обучающего набора**
Для каждой пары создаются:
- One-hot векторы для центрального слова и контекстного слова.
- Негативные примеры (слова, которые не являются контекстом).

#### **3.5. Реализация моделей**
1. **CBOW**:
   - Вход: контекстные слова.
   - Выход: центральное слово.
   - Использует два слоя (матрицы весов): первый слой преобразует one-hot векторы в эмбеддинги, второй слой преобразует эмбеддинги обратно в вероятности слов.
2. **SGNS**:
   - Вход: центральное слово.
   - Выход: контекстные слова.
   - Использует два слоя (матрицы весов): первый слой преобразует one-hot векторы в эмбеддинги, второй слой преобразует эмбеддинги в вероятности слов. Также минимизируется ошибка на негативных примерах.

#### **3.6. Обучение моделей**
Модели обучаются с использованием градиентного спуска. Ошибка уменьшается на каждой итерации, что визуализируется с помощью графика.

#### **3.7. Поиск похожих слов**
После обучения модели можно найти слова, которые имеют схожие эмбеддинги. Например, для слова "learning" можно найти слова, которые часто встречаются в похожих контекстах.

---

### **4. Важные аспекты**

1. **CBOW и Skip-gram**: Два подхода к обучению Word2Vec. CBOW быстрее, но Skip-gram лучше работает с редкими словами.
2. **Негативное сэмплирование**: Ускоряет обучение, минимизируя ошибку только на небольшом числе негативных примеров.
3. **Эмбеддинги**: Векторные представления слов, которые можно использовать для поиска похожих слов, кластеризации и других задач.
4. **Косинусное сходство**: Метрика для измерения схожести между векторами.

---



In [ ]:
import regex as re
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict

import spacy
nlp = spacy.load("en_core_web_sm")


In [ ]:
text = open("./data/ml_text.txt").read()
print(text[0:150], "...")

Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from ...


In [ ]:
def tokenize_paragrah(paragraph, lemmaOn=False):
    """Build list of sentences and list of tokens for each sentence.
        Punctuation is removed
        If lemmatization is applied, all the tokens are lemmatized first
    """
    doc = nlp(paragraph)
    tokenized_text = []
    for sent in doc.sents:
        if lemmaOn:
            tokenized_text.append([token.lemma_ for token in sent if not token.is_punct])
        else:
            tokenized_text.append([token.text.lower() for token in sent if not token.is_punct])
    return tokenized_text


# def build_bag_of_words(tokenized_paragraph):
#     """Builds two dictionaries that map token into index and vice versa
#     """
#     bow = set()
#     for sent in tokenized_paragraph:
#         for token in sent:
#             bow.add(token)

#     token2id = {}
#     id2token = {}
#     for i, t in enumerate(bow):
#         id2token[i+1] = t
#         token2id[t] = i+1

#     token2id["unk"] = 0
#     id2token[0] = "unk"
#     return id2token, token2id

def build_vocab(tokenized_paragraph):
    word_freq = defaultdict(int)
    for sent in tokenized_paragraph:
        for token in sent:
            word_freq[token] += 1
    word2idx = {word: idx for idx, word in enumerate(word_freq)}
    idx2word = {idx: word for word, idx in word2idx.items()}
    counts = np.array([word_freq[idx2word[i]] for i in range(len(word2idx))], dtype=np.float32)
    freq_table = counts ** 0.75
    freq_table /= freq_table.sum()
    return idx2word, word2idx, freq_table

def one_hot_encode(idx, vocab_size):
    """Converts token id into one-hot vector
    """
    res = [0] * vocab_size
    res[idx] = 1
    return res

def generate_tokens_pairs(sentence, window=2):
    """Generates list of pairs: (central token, token in window-neiborhood of central token)
    """
    tokens_pairs = []
    n_tokens = len(sentence)

    for i, c_token in enumerate(sentence):
        for idx in range(max(0, i - window), i):
            tokens_pairs.append((c_token, sentence[idx]))
        for idx in range(i+1, min(n_tokens, i + window + 1)):
            tokens_pairs.append((c_token, sentence[idx]))
    return tokens_pairs

def get_negative_samples(vocab_size, neg_sample_size, freq_table):
    return np.random.choice(np.arange(vocab_size), size=neg_sample_size, p=freq_table)

def build_training_set(tokens_pairs, token2id, freq_table, neg_sample_size=5):
    X = []
    y = []
    neg_y = []
    for x_token, y_token in tokens_pairs:
        X.append(one_hot_encode(token2id[x_token], len(token2id)))
        y.append(one_hot_encode(token2id[y_token], len(token2id)))
        neg_ids = get_negative_samples(len(token2id), neg_sample_size, freq_table)
        # for i in neg_ids:
        #     print(i, tokens_pairs[i][1])
        #print(neg_ids)
        neg_y.append([one_hot_encode(token2id[tokens_pairs[i][1]], len(token2id)) for i in neg_ids])
    return np.array(X), np.array(y), np.array(neg_y)



In [ ]:
tokenized_text = tokenize_paragrah(text)
#id2token, token2id = build_bag_of_words(tokenized_text)
id2token, token2id, freq_table = build_vocab(tokenized_text)
print(token2id)

vocab_size = len(id2token)
print(vocab_size)

print(tokenized_text[0])
tokens_pairs = generate_tokens_pairs(tokenized_text[0])
token_pairs = []
for sentence in tokenized_text:
    tokens_pairs.extend(generate_tokens_pairs(sentence))
train_X, train_y, train_neg_y = build_training_set(tokens_pairs, token2id, freq_table)

print(train_X.shape)

{'machine': 0, 'learning': 1, 'is': 2, 'a': 3, 'field': 4, 'of': 5, 'study': 6, 'in': 7, 'artificial': 8, 'intelligence': 9, 'concerned': 10, 'with': 11, 'the': 12, 'development': 13, 'and': 14, 'statistical': 15, 'algorithms': 16, 'that': 17, 'can': 18, 'learn': 19, 'from': 20, 'data': 21, 'generalize': 22, 'to': 23, 'unseen': 24, 'thus': 25, 'perform': 26, 'tasks': 27, 'without': 28, 'explicit': 29, 'instructions': 30, 'within': 31, 'subdiscipline': 32, 'advances': 33, 'deep': 34, 'have': 35, 'allowed': 36, 'neural': 37, 'networks': 38, 'class': 39, 'surpass': 40, 'many': 41, 'previous': 42, 'approaches': 43, 'performance': 44, 'ml': 45, 'finds': 46, 'application': 47, 'fields': 48, 'including': 49, 'natural': 50, 'language': 51, 'processing': 52, 'computer': 53, 'vision': 54, 'speech': 55, 'recognition': 56, 'email': 57, 'filtering': 58, 'agriculture': 59, 'medicine': 60, 'business': 61, 'problems': 62, 'known': 63, 'as': 64, 'predictive': 65, 'analytics': 66, 'statistics': 67, 'mat

In [ ]:
print("vocabulary position:", tokens_pairs[0], token2id[tokens_pairs[0][0]], token2id[tokens_pairs[0][1]])
(train_X[0], train_y[0], train_neg_y[0])
print("one-hot indices:", train_X[0].argmax(), train_y[0].argmax())
for i in train_neg_y[0]:
    print(id2token[i.argmax()])

vocabulary position: ('machine', 'learning') 0 1
one-hot indices: 0 1
machine
artificial
field
mirror
to


In [ ]:
import torch
from torch import nn, optim
from torch.nn.functional import logsigmoid
from tqdm import  tqdm

In [ ]:
class CBOW(nn.Module):
    """
    Continuous bag of words
    """

    def __init__(self, vocab_size, n_embeddings) -> None:
        super().__init__()

        self.vocab_size = vocab_size
        self.vector_dim = n_embeddings
        self.W1 = nn.Parameter(data=torch.randn(self.vocab_size, self.vector_dim), requires_grad=True) # Word Vectors
        self.W2 = nn.Parameter(data=torch.randn(self.vector_dim, self.vocab_size), requires_grad=True)

    def forward(self, X) -> torch.tensor:
        X = X @ self.W1
        X = X @ self.W2
        return X

In [ ]:
class SGNS(nn.Module):
    """
    Skip-gram negative sampling
    """

    def __init__(self, vocab_size, embedding_dim):
        super(SGNS, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        # Initialize embedding matrices as parameters
        self.input_weights = nn.Parameter(torch.randn(vocab_size, embedding_dim) * 0.01, requires_grad=True)
        self.output_weights = nn.Parameter(torch.randn(vocab_size, embedding_dim) * 0.01, requires_grad=True)

    def forward(self, center_onehot, pos_onehot, neg_onehots):
        # Convert one-hot to embeddings via matrix multiply
        v_c = center_onehot @ self.input_weights  # (batch_size, embedding_dim)
        u_o = pos_onehot @ self.output_weights    # (batch_size, embedding_dim)
        u_k = neg_onehots @ self.output_weights   # (batch_size, neg_samples, embedding_dim)

        # Positive loss: log σ(uᵀv)
        pos_score = torch.sum(v_c * u_o, dim=1)
        pos_loss = logsigmoid(pos_score)

        # Negative loss: Σ log σ(-uₖᵀv)
        neg_score = torch.bmm(u_k.neg(), v_c.unsqueeze(2)).squeeze()
        neg_loss = logsigmoid(neg_score).sum(1)

        loss = -(pos_loss + neg_loss).mean()
        return loss

    def get_embeddings(self):
        return self.input_weights.data

# def get_negative_samples(batch_size, vocab_size, neg_sample_size, freq_table):
#     return np.random.choice(np.arange(vocab_size), size=(batch_size, neg_sample_size), p=freq_table)

In [ ]:
x = torch.tensor(train_X, dtype=torch.float)
print(x.size())
y = torch.tensor(train_y, dtype=torch.float)
print(y.size())
y_neg = torch.tensor(train_neg_y, dtype=torch.float)
print(y_neg.size())

torch.Size([30482, 2043])
torch.Size([30482, 2043])
torch.Size([30482, 5, 2043])


In [ ]:
eps = 0.1
vocab_size = len(id2token)
n_embeddings = 10

cbow_model = CBOW(vocab_size, n_embeddings)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=cbow_model.parameters(), lr=eps)

error = []
for epoch in tqdm(range(100)):
    pred = cbow_model(x)
    train_loss = loss_fn(pred, y)
    error.append(train_loss.item())

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

fig = go.Figure(data=[go.Scatter(y=error)])
fig.show()

100%|██████████| 100/100 [00:54<00:00,  1.83it/s]


In [ ]:
eps = 0.1
vocab_size = len(id2token)
n_embeddings = 10
error = []

sgns_model = SGNS(vocab_size, n_embeddings)
optimizer = torch.optim.Adam(params=sgns_model.parameters(), lr=eps)

for epoch in tqdm(range(100)):
    loss = sgns_model(x, y, y_neg)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #print(f"Epoch {epoch+1}, Loss: {loss}")
    error.append(loss.item())

fig = go.Figure(data=[go.Scatter(y=error)])
fig.show()

100%|██████████| 100/100 [01:20<00:00,  1.25it/s]


In [ ]:
params = list(sgns_model.parameters())
word_vectors = params[0].detach()
print(word_vectors.size())

token2vec_sgns = {t: word_vectors[idx] for t, idx in token2id.items()}
token2vec_sgns

torch.Size([2043, 10])


{'machine': tensor([ 0.3810,  0.7502, -0.6660,  0.4690,  0.3045,  0.2329, -0.6503, -0.3466,
          0.8578, -0.5500]),
 'learning': tensor([ 0.2273,  0.4385, -0.5234,  0.2622,  0.4667, -0.3320, -0.7892, -0.1773,
          0.7121, -0.7949]),
 'is': tensor([ 0.6379,  0.7636, -0.0579,  0.5094,  1.2126, -0.0951, -0.4526, -0.1345,
          0.5133, -0.4501]),
 'a': tensor([ 0.3818,  0.5863, -0.8124, -0.1926,  0.5123,  0.2469, -0.2554, -0.2236,
          0.4293, -0.7129]),
 'field': tensor([ 0.8555,  0.8479, -0.3803,  0.4819,  0.5432,  0.6659, -0.0064,  0.2588,
          0.3657, -0.0637]),
 'of': tensor([ 0.5286,  0.5603, -0.4893,  0.3265,  1.2257,  0.2370, -0.5324,  0.1475,
          0.7003,  0.1440]),
 'study': tensor([ 0.7178,  0.7882, -0.8243, -0.0430,  0.7416,  1.1370, -0.0074,  0.2076,
          0.3919,  0.7410]),
 'in': tensor([ 0.3781,  0.6352, -0.7458,  0.1710,  0.7338, -0.1881, -0.5692, -0.0640,
          0.5514,  0.0048]),
 'artificial': tensor([ 0.6958, -0.3487, -0.8006, -0.075

In [ ]:
params = list(cbow_model.parameters())
word_vectors = params[0].detach()
print(word_vectors.size())

token2vec_cbow = {t: word_vectors[idx] for t, idx in token2id.items()}
token2vec_cbow

torch.Size([2043, 10])


{'machine': tensor([-3.5331e-01, -7.7664e-01,  3.9965e-01,  6.0956e-04, -9.8289e-02,
          6.5419e-01, -7.7562e-01, -3.6816e-01,  1.1995e+00,  3.5494e-01]),
 'learning': tensor([-0.3645, -0.7337,  0.5741,  0.1238, -0.0712,  0.4771, -0.4819, -0.2278,
          0.8965,  0.5596]),
 'is': tensor([-0.0156, -0.1883,  0.6597,  0.2720,  0.0811,  0.1547, -0.4673,  0.4984,
          0.6780,  0.0655]),
 'a': tensor([-0.1291, -0.2101,  0.3740,  0.0362,  0.0497,  0.2476, -0.0875,  0.2451,
          0.8629,  0.1227]),
 'field': tensor([-0.1750,  0.2042,  0.2556, -0.6340, -0.3357,  0.2058, -0.7190,  0.4482,
          1.7562, -0.4041]),
 'of': tensor([-0.3973, -0.2825,  0.2851, -0.5136,  0.1257, -0.1950, -0.3466,  0.1441,
          0.7236, -0.2274]),
 'study': tensor([ 0.0500, -0.4144, -0.1152, -1.3730,  0.0061,  0.6234, -1.7757, -0.4184,
          1.5606, -0.6377]),
 'in': tensor([-6.1641e-01,  2.1985e-01,  5.5948e-01, -2.2816e-01,  5.4762e-04,
          8.8779e-02, -5.0128e-01,  1.0189e-01,  4.6

In [ ]:
def cosine_similarity(v1, v2):
    return (v1 @ v2) / (torch.norm(v1) * torch.norm(v2))

def most_similar(word, word_dict, top_k=5):
    if word not in word_dict:
        raise ValueError(f"{word} not found in the word dictionary.")

    query_vector = word_dict[word]

    # Calculate cosine similarity with all other words in the dictionary
    similarities = {}
    for other_word, other_vector in word_dict.items():
        if other_word != word:
            similarity = cosine_similarity(query_vector, other_vector)
            similarities[other_word] = similarity

    # Sort the words by similarity in descending order
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    # Get the top-k most similar words
    top_similar_words = sorted_similarities[:top_k]

    return top_similar_words

In [ ]:
most_similar("learning", token2vec_cbow)


[('machine', tensor(0.9608)),
 ('unsupervised', tensor(0.9042)),
 ('reinforcement', tensor(0.9034)),
 ('self', tensor(0.8971)),
 ('workloads', tensor(0.8866))]

In [ ]:
most_similar("learning", token2vec_sgns)

[('semi', tensor(0.9612)),
 ('relevant', tensor(0.9280)),
 ('graduates', tensor(0.9254)),
 ('accordingly', tensor(0.9239)),
 ('approach&#91;clarification', tensor(0.9229))]

In [ ]:
## try different embedding sizes and windows and make conclusions